In [ ]:
import redis
import os
from neptune_python_utils.gremlin_utils import GremlinUtils
import ipywidgets as widgets
from IPython.display import display, HTML

### Create Neptune client

In [ ]:
GremlinUtils.init_statics(globals())

gremlin = GremlinUtils()

neptune_connection = gremlin.remote_connection()
g = gremlin.traversal_source(connection=neptune_connection)

### Create ElastiCache client

In [ ]:
ec = redis.StrictRedis(host=os.environ['ELASTICACHE_ENDPOINT'], port=6379)

### Query ElastiCache and Neptune

In [ ]:
class HtmlTable:
    def __init__(self):
        self.html = '<table style="border: 1px solid black">'
        self.empty_row = '<tr style="border: 1px solid black;"><td style="border: 1px solid black; vertical-align: top; text-align: center;" colspan="4">No results</td></tr>'
    
    def add_header_row(self, cols):
        self.html += '<tr style="border: 1px solid black; background-color: gainsboro; font-weight: bold;">'
        for col in cols:
            self.html += '<td style="border: 1px solid black; vertical-align: top; text-align: left;">{}</td>'.format(col)
        self.html += '</tr>'
        
    def add_row(self, cols):
        self.empty_row = ''
        self.html += '<tr style="border: 1px solid black;">'
        for col in cols:
            self.html += '<td style="border: 1px solid black; vertical-align: top; text-align: left;">{}</td>'.format(col)
        self.html += '</tr>'
        
    def get_html(self):
        result = self.html + self.empty_row + '</table>'
        return result
        
most_popular = ec.zrevrange('degree_centrality', 0, 2, withscores=True)

vertex_ids = []
centralities = []

for (vertex_id, centrality) in most_popular:
    vertex_ids.append(vertex_id.decode("utf-8") )
    centralities.append(int(centrality))
    
users = g.V(vertex_ids).valueMap(True).toList()

tbl = HtmlTable()
tbl.add_header_row(['First Name', 'Last Name', 'Age', 'Centrality'])

for (user, centrality) in zip(users, centralities):
    tbl.add_row([user['firstname'][0], user['lastname'][0], user['age'][0], centrality])
    
display(HTML(tbl.get_html()))

### Close Neptune client

In [ ]:
neptune_connection.close()